# Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
library(brms)
theme_set(theme_bw())
# helpers_path = paste0(here(),'/analysis/helpers/')
helpers_path = here()
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_bc_fix = read.csv(paste0(here(), '/inputs/data_fixBC.csv'))

data_bc_fix = data_bc_fix %>%
  mutate(correct = ifelse(possiblePayoffleft>possiblePayoffright & leftChosen == 1, 1, ifelse(possiblePayoffleft<possiblePayoffright & leftChosen == 0, 1, 0))) %>%
  mutate(type = ifelse(typeLeft == 1, "HT", "RE")) %>%
  mutate(val_diff = possiblePayoffleft - possiblePayoffright,
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-275:-225","-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225", "225:275")),
         abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275"))) %>%
  filter(!is.na(val_diff_bin_str))

# Fixation change models

## Fix duration

### Linear

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_fixdur_linear_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_fix_dur_linear_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_fixdur_linear_draws = read.csv(fn_draws)
  bc_fixdur_linear_ppc = read.csv(fn_ppc)
  
} else{
  subnums = unique(data_bc_fix$subnum)
  
  bc_fixdur_linear_draws = data_frame()
  bc_fixdur_linear_ppc = data_frame()
  
  data_bc_fix_cut = data_bc_fix %>%
    select(subnum, day, type, firstFix, middleFix, lastFix, fixDuration) %>%
    mutate(fixTypeNum = ifelse(firstFix == 1, 1, ifelse(middleFix == 1, 2, ifelse(lastFix == 1, 3, NA)))) %>%
    select(-firstFix, -middleFix, -lastFix)
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>%
      ungroup() %>%
      filter(subnum == cur_sub)

    cur_bc_fixdur_linear <- brm(formula = fixDuration ~ fixTypeNum * day * type,
                        data= cur_bc_fix_data,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_fixdur_linear)
    cur_draws$subnum = cur_sub
    
    bc_fixdur_linear_draws = rbind(bc_fixdur_linear_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_fixdur_linear, newdata = cur_bc_fix_data %>%
                        select(fixTypeNum, day, type), scale = "linear") %>%
      as_tibble() %>%
      bind_cols(cur_bc_fix_data)
    
    bc_fixdur_linear_ppc = rbind(bc_fixdur_linear_ppc, cur_pred)
  }
  names(bc_fixdur_linear_draws) = gsub(":", ".", names(bc_fixdur_linear_draws))
  write.csv(bc_fixdur_linear_draws, fn_draws, row.names = F)
  write.csv(bc_fixdur_linear_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_fixdur_linear, cur_draws, cur_pred)
}

In [ ]:
bc_par_summary = as.data.frame(bc_fixdur_linear_draws) %>%
  select("b_fixTypeNum", "b_fixTypeNum.day", "b_fixTypeNum.typeRE", "b_fixTypeNum.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_fixTypeNum", "Fixation Type", ifelse(key == "b_fixTypeNum.day", "Fixation Type * Day", ifelse(key == "b_fixTypeNum.typeRE", "Fixation Type * Type (RE)", ifelse(key == "b_fixTypeNum.day.typeRE", "Fixation Type * Day * Type (RE)", NA)))),
         key = factor(key, levels = c("Fixation Type", "Fixation Type * Type (RE)", "Fixation Type * Day", "Fixation Type * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Fixation Duration ~ Fixation Type * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_fixDurationLin_interactionParEsts.jpg'), p, height = 3, width = 9, units="in")

### Quadratic + quadratic interactions

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_fixdur_quadratic_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_fix_dur_quadratic_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_fixdur_quadratic_draws = read.csv(fn_draws)
  bc_fixdur_quadratic_ppc = read.csv(fn_ppc)
  
} else{
  subnums = unique(data_bc_fix$subnum)
  
  bc_fixdur_quadratic_draws = data_frame()
  bc_fixdur_quadratic_ppc = data_frame()
  
  data_bc_fix_cut = data_bc_fix %>%
    select(subnum, day, type, firstFix, middleFix, lastFix, fixDuration) %>%
    mutate(fixTypeNum = ifelse(firstFix == 1, 1, ifelse(middleFix == 1, 2, ifelse(lastFix == 1, 3, NA))),
           fixTypeNum2 = fixTypeNum^2) %>%
    select(-firstFix, -middleFix, -lastFix)
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>%
      ungroup() %>%
      filter(subnum == cur_sub)

    cur_bc_fixdur_quadratic <- brm(formula = fixDuration ~ fixTypeNum + fixTypeNum2 * day * type,
                        data= cur_bc_fix_data,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_fixdur_quadratic)
    cur_draws$subnum = cur_sub
    
    bc_fixdur_quadratic_draws = rbind(bc_fixdur_quadratic_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_fixdur_quadratic, newdata = cur_bc_fix_data %>%
                        select(fixTypeNum, fixTypeNum2, day, type)) %>%
      as_tibble() %>%
      bind_cols(cur_bc_fix_data)
    
    bc_fixdur_quadratic_ppc = rbind(bc_fixdur_quadratic_ppc, cur_pred)
  }
  names(bc_fixdur_quadratic_draws) = gsub(":", ".", names(bc_fixdur_quadratic_draws))
  write.csv(bc_fixdur_quadratic_draws, fn_draws, row.names = F)
  write.csv(bc_fixdur_quadratic_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_fixdur_quadratic, cur_draws, cur_pred)
}

In [ ]:
bc_par_summary = as.data.frame(bc_fixdur_quadratic_draws) %>%
  select("b_fixTypeNum", "b_fixTypeNum2","b_fixTypeNum2.day", "b_fixTypeNum2.typeRE", "b_fixTypeNum2.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_fixTypeNum", "Fixation Type", ifelse(key == "b_fixTypeNum2.day", "Fixation Type^2 * Day", ifelse(key == "b_fixTypeNum2.typeRE", "Fixation Type^2 * Type (RE)", ifelse(key == "b_fixTypeNum2.day.typeRE", "Fixation Type^2 * Day * Type (RE)", ifelse(key == "b_fixTypeNum2", "Fixation Type^2", NA))))),
         key = factor(key, levels = c("Fixation Type", "Fixation Type^2","Fixation Type^2 * Type (RE)", "Fixation Type^2 * Day", "Fixation Type^2 * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Fixation Duration ~ Fixation Type + Fixation Type^2 * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_fixDurationQuad_interactionParEsts.jpg'), p, height = 3, width = 11, units="in")

### Quadratic + linear interactions 

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_fixdur_quadratic2_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_fix_dur_quadratic2_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_fixdur_quadratic_draws = read.csv(fn_draws)
  bc_fixdur_quadratic_ppc = read.csv(fn_ppc)
  
} else{
  subnums = unique(data_bc_fix$subnum)
  
  bc_fixdur_quadratic_draws = data_frame()
  bc_fixdur_quadratic_ppc = data_frame()
  
  data_bc_fix_cut = data_bc_fix %>%
    select(subnum, day, type, firstFix, middleFix, lastFix, fixDuration) %>%
    mutate(fixTypeNum = ifelse(firstFix == 1, 1, ifelse(middleFix == 1, 2, ifelse(lastFix == 1, 3, NA))),
           fixTypeNum2 = fixTypeNum^2) %>%
    select(-firstFix, -middleFix, -lastFix)
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>%
      ungroup() %>%
      filter(subnum == cur_sub)

    cur_bc_fixdur_quadratic <- brm(formula = fixDuration ~ fixTypeNum2 + fixTypeNum * day * type,
                        data= cur_bc_fix_data,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_fixdur_quadratic)
    cur_draws$subnum = cur_sub
    
    bc_fixdur_quadratic_draws = rbind(bc_fixdur_quadratic_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_fixdur_quadratic, newdata = cur_bc_fix_data %>%
                        select(fixTypeNum, fixTypeNum2, day, type)) %>%
      as_tibble() %>%
      bind_cols(cur_bc_fix_data)
    
    bc_fixdur_quadratic_ppc = rbind(bc_fixdur_quadratic_ppc, cur_pred)
  }
  names(bc_fixdur_quadratic_draws) = gsub(":", ".", names(bc_fixdur_quadratic_draws))
  write.csv(bc_fixdur_quadratic_draws, fn_draws, row.names = F)
  write.csv(bc_fixdur_quadratic_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_fixdur_quadratic, cur_draws, cur_pred)
}

In [ ]:
bc_par_summary = as.data.frame(bc_fixdur_quadratic_draws) %>%
  select("b_fixTypeNum", "b_fixTypeNum2","b_fixTypeNum.day", "b_fixTypeNum.typeRE", "b_fixTypeNum.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_fixTypeNum", "Fixation Type", ifelse(key == "b_fixTypeNum.day", "Fixation Type * Day", ifelse(key == "b_fixTypeNum.typeRE", "Fixation Type * Type (RE)", ifelse(key == "b_fixTypeNum.day.typeRE", "Fixation Type * Day * Type (RE)", ifelse(key == "b_fixTypeNum2", "Fixation Type^2", NA))))),
         key = factor(key, levels = c("Fixation Type", "Fixation Type^2","Fixation Type * Type (RE)", "Fixation Type * Day", "Fixation Type * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Fixation Duration ~ Fixation Type^2 + Fixation Type * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_fixDurationQuad2_interactionParEsts.jpg'), p, height = 3, width = 11, units="in")

## Number of fixations

number of fixations ~ abs_value difference * day * type

In [ ]:
data_bc_fix %>%
  filter(lastFix != 1) %>%
  group_by(subnum, day, trialNum) %>%
  summarise(type = unique(type),
            abs_val_diff = unique(abs(val_diff)), 
            numTotalFix = unique(numTotalFix),
            .groups = 'keep')

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_numfix_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_numfix_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_numfix_draws = read.csv(fn_draws)
  bc_numfix_ppc = read.csv(fn_ppc)
} else{
  subnums = unique(data_bc_fix$subnum)
  
  bc_numfix_draws = data_frame()
  bc_numfix_ppc = data_frame()
  
  data_bc_fix_cut = data_bc_fix %>%
    filter(lastFix != 1) %>%
    group_by(subnum, day, trialNum) %>%
    summarise(type = unique(type),
              abs_val_diff = unique(abs(val_diff)), 
              numTotalFix = unique(numTotalFix),
              .groups = 'keep')
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    cur_bc_numfix <- brm(formula = numTotalFix ~ abs_val_diff * day * type,
                     data= cur_bc_fix_data,
                     warmup = 500,
                     iter = 2500,
                     chains = 4,
                     init = "0",
                     cores = 4,
                     seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_numfix)
    cur_draws$subnum = cur_sub
    
    bc_numfix_draws = rbind(bc_numfix_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_numfix, newdata = cur_bc_fix_data) %>% 
      as_tibble() %>%
      # transmute(pred_log_rt = Estimate) %>% #same as mean brms::posterior_epred values not brms::posterior_predict
      bind_cols(cur_bc_fix_data) %>%
      mutate(subnum = cur_sub)
    
    bc_numfix_ppc = rbind(bc_numfix_ppc, cur_pred)
  }
  names(bc_numfix_draws) = gsub(":", ".", names(bc_numfix_draws))
  write.csv(bc_numfix_draws, fn_draws, row.names = F)
  write.csv(bc_numfix_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_numfix, cur_draws, cur_pred)
}

In [ ]:
bc_par_summary = as.data.frame(bc_numfix_draws) %>%
  select("b_abs_val_diff", "b_abs_val_diff.day", "b_abs_val_diff.typeRE", "b_abs_val_diff.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_abs_val_diff", "|Value Difference|", ifelse(key == "b_abs_val_diff.day", "|Val Diff| * Day", ifelse(key == "b_abs_val_diff.typeRE", "|Val Diff| * Type (RE)", ifelse(key == "b_abs_val_diff.day.typeRE", "|Val Diff| * Day * Type (RE)", NA)))),
         key = factor(key, levels = c("|Value Difference|", "|Val Diff| * Type (RE)", "|Val Diff| * Day", "|Val Diff| * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Number of Fixations ~ |Value Difference| * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_numFixOverAbsValDiff_interactionParEsts.jpg'), p, height = 3, width = 9, units="in")

## First fixation location

logit(first fixation == left) ~ value difference * day * type

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_firstFixLoc_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_firstFixLoc_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_firstFixLoc_draws = read.csv(fn_draws)
  bc_firstFixLoc_ppc = read.csv(fn_ppc)
} else{
  
  data_bc_fix_cut = data_bc_fix %>%
    filter(firstFix == 1) %>%
    select(subnum, day, leftFix, val_diff, type)
  
  subnums = unique(data_bc_fix_cut$subnum)
  
  bc_firstFixLoc_draws = data_frame()
  bc_firstFixLoc_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    
    cur_bc_firstFixLoc <- brm(formula = leftFix ~ val_diff * day * type,
                        data= cur_bc_fix_data,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_firstFixLoc)
    cur_draws$subnum = cur_sub
    
    bc_firstFixLoc_draws = rbind(bc_firstFixLoc_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_firstFixLoc, newdata = cur_bc_fix_data, scale = "linear") %>%
      as_tibble() %>%
      transmute(prob = Estimate %>% inv_logit_scaled()) %>%
      bind_cols(cur_bc_fix_data) %>%
      mutate(subnum = cur_sub)
    
    bc_firstFixLoc_ppc = rbind(bc_firstFixLoc_ppc, cur_pred)
  }
  names(bc_firstFixLoc_draws) = gsub(":", ".", names(bc_firstFixLoc_draws))
  write.csv(bc_firstFixLoc_draws, fn_draws, row.names = F)
  write.csv(bc_firstFixLoc_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_firstFixLoc, cur_draws, cur_pred)
}


In [ ]:
bc_par_summary = as.data.frame(bc_firstFixLoc_draws) %>%
  select("b_val_diff", "b_val_diff.day", "b_val_diff.typeRE", "b_val_diff.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff", "Value Difference", ifelse(key == "b_val_diff.day", "Val Diff * Day", ifelse(key == "b_val_diff.typeRE", "Val Diff * Type (RE)", ifelse(key == "b_val_diff.day.typeRE", "Val Diff * Day * Type (RE)", NA)))),
         key = factor(key, levels = c("Value Difference", "Val Diff * Type (RE)", "Val Diff * Day", "Val Diff * Day * Type (RE)"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - First Fixation == Left ~ Value Difference * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
ggsave(file=paste0(fig_out_path, 'bc_firstFixLocsValDiff_interactionParEsts.jpg'), p, height = 3, width = 9, units="in")

## First fixation effect on choice

logit(choice == left) ~ leftFix * value difference * day * type

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_firstFixLogit_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_firstFixLogit_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_firstFixLogit_draws = read.csv(fn_draws)
  bc_firstFixLogit_ppc = read.csv(fn_ppc)
} else{
  
  data_bc_fix_cut = data_bc_fix %>%
    filter(!is.na(val_diff_bin_str)) %>%
    filter(firstFix == 1) %>%
    select(subnum, leftFix, leftChosen, day, type, val_diff)
  
  subnums = unique(data_bc_fix_cut$subnum)
  
  bc_firstFixLogit_draws = data_frame()
  bc_firstFixLogit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    
    cur_bc_firstFixLogit <- brm(formula = leftChosen ~ leftFix * val_diff * day * type,
                        data= cur_bc_fix_data,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_firstFixLogit)
    cur_draws$subnum = cur_sub
    
    bc_firstFixLogit_draws = rbind(bc_firstFixLogit_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_firstFixLogit, newdata = cur_bc_fix_data , scale = "linear") %>%
      as_tibble() %>%
      transmute(prob = Estimate %>% inv_logit_scaled()) %>%
      bind_cols(cur_bc_fix_data) %>%
      mutate(subnum = cur_sub)
    
    bc_firstFixLogit_ppc = rbind(bc_firstFixLogit_ppc, cur_pred)
  }
  names(bc_firstFixLogit_draws) = gsub(":", ".", names(bc_firstFixLogit_draws))
  write.csv(bc_firstFixLogit_draws, fn_draws, row.names = F)
  write.csv(bc_firstFixLogit_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_firstFixLogit, cur_draws, cur_pred)
}


In [ ]:
bc_par_summary = as.data.frame(bc_firstFixLogit_draws) %>%
  select("b_val_diff", "b_val_diff.day", "b_val_diff.typeRE", "b_val_diff.day.typeRE", "b_leftFix.val_diff", "b_leftFix.val_diff.day", "b_leftFix.val_diff.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff", "Value Difference", ifelse(key == "b_val_diff.day", "Val Diff * Day", ifelse(key == "b_val_diff.typeRE", "Val Diff * Type (RE)", ifelse(key == "b_val_diff.day.typeRE", "Val Diff * Day * Type (RE)", ifelse(key == "b_leftFix.val_diff", "Val Diff * First Fixation", ifelse(key == "b_leftFix.val_diff.day", "Val Diff * First Fixation * Day", ifelse(key == "b_leftFix.val_diff.day.typeRE", "4 Way", NA))))))),
         key = factor(key, levels = c("Value Difference", "Val Diff * Type (RE)", "Val Diff * Day", "Val Diff * First Fixation", "Val Diff * Day * Type (RE)", "Val Diff * First Fixation * Day", "4 Way"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Choice == Left ~ First Fixation * Value Difference * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_firstFixLogit_interactionParEsts.jpg'), p, height = 3, width = 15, units="in")

## Last fixation effect on choice

logit(choice == left) ~ last fixation side * value difference * day * type

In [ ]:
fn_draws = paste0(helpers_path, '/inputs/bc_lastFixLogit_draws.csv')
fn_ppc = paste0(helpers_path, '/inputs/bc_lastFixLogit_ppc.csv')

if(file.exists(fn_draws) & file.exists(fn_ppc)){
  bc_lastFixLogit_draws = read.csv(fn_draws)
  bc_lastFixLogit_ppc = read.csv(fn_ppc)
} else{
  
  data_bc_fix_cut = data_bc_fix %>%
    filter(!is.na(val_diff_bin_str)) %>%
    filter(lastFix == 1) %>%
    select(subnum, leftFix, leftChosen, day, type, val_diff)
  
  subnums = unique(data_bc_fix_cut$subnum)
  
  bc_lastFixLogit_draws = data_frame()
  bc_lastFixLogit_ppc = data_frame()
  
  for(i in 1:length(subnums)){
    cur_sub = subnums[i]
    
    cur_bc_fix_data = data_bc_fix_cut %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    
    cur_bc_lastFixLogit <- brm(formula = leftChosen ~ leftFix * val_diff * day * type,
                        data= cur_bc_fix_data,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_bc_lastFixLogit)
    cur_draws$subnum = cur_sub
    
    bc_lastFixLogit_draws = rbind(bc_lastFixLogit_draws, cur_draws)
    
    cur_pred = fitted(cur_bc_lastFixLogit, newdata = cur_bc_fix_data , scale = "linear") %>%
      as_tibble() %>%
      transmute(prob = Estimate %>% inv_logit_scaled()) %>%
      bind_cols(cur_bc_fix_data) %>%
      mutate(subnum = cur_sub)
    
    bc_lastFixLogit_ppc = rbind(bc_lastFixLogit_ppc, cur_pred)
  }
  names(bc_lastFixLogit_draws) = gsub(":", ".", names(bc_lastFixLogit_draws))
  write.csv(bc_lastFixLogit_draws, fn_draws, row.names = F)
  write.csv(bc_lastFixLogit_ppc, fn_ppc, row.names = F)
  
  rm(cur_bc_fix_data, cur_bc_lastFixLogit, cur_draws, cur_pred)
}


In [ ]:
bc_par_summary = as.data.frame(bc_lastFixLogit_draws) %>%
  select("b_val_diff", "b_val_diff.day", "b_val_diff.typeRE", "b_val_diff.day.typeRE", "b_leftFix.val_diff", "b_leftFix.val_diff.day", "b_leftFix.val_diff.day.typeRE", subnum) %>%
  gather(key, value, -subnum) %>%
  mutate(key = ifelse(key == "b_val_diff", "Value Difference", ifelse(key == "b_val_diff.day", "Val Diff * Day", ifelse(key == "b_val_diff.typeRE", "Val Diff * Type (RE)", ifelse(key == "b_val_diff.day.typeRE", "Val Diff * Day * Type (RE)", ifelse(key == "b_leftFix.val_diff", "Val Diff * First Fixation", ifelse(key == "b_leftFix.val_diff.day", "Val Diff * First Fixation * Day", ifelse(key == "b_leftFix.val_diff.day.typeRE", "4 Way", NA))))))),
         key = factor(key, levels = c("Value Difference", "Val Diff * Type (RE)", "Val Diff * Day", "Val Diff * First Fixation", "Val Diff * Day * Type (RE)", "Val Diff * First Fixation * Day", "4 Way"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum),
         task = "BC") 

In [ ]:
plim = max(abs(bc_par_summary$l95),bc_par_summary$h95)

p = bc_par_summary %>%
  ggplot(aes(m, subnum), color = "red") +
  geom_vline(aes(xintercept = 0), color = "light gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_grid(.~key) +
  labs(x = "Parameter Estimate", y = "Subject", title = "95% HDI for posterior slopes - Last Fixation == Left ~ Value Difference * Day * Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_continuous(breaks = c(0), limits = c(-plim, plim))

p
# ggsave(file=paste0(fig_out_path, 'bc_lastFixLogit_interactionParEsts.jpg'), p, height = 3, width = 15, units="in")